In [1]:
import numpy as np
from scipy.spatial.distance import cdist

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy.linalg import eigh
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

# hyperparameters: Kernel, Regularization, Gamma

In [27]:
# Kernel function

def rbf_kernel(X, sigma, diag=1):
    # Basic SVM with predefined kernel matrix
    N = X.shape[0]
    K = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            if i == j:
                K[i,j] = diag
            else:
                x_i = X[i, :].reshape(1,-1)
                x_j = X[j, :].reshape(1,-1)

                K[i, j] = np.exp(-cdist(x_i, x_j, 'sqeuclidean') / (sigma ** 2)) # euclidean?
    return K


def make_D_matrix(K):
    K_sum = np.sum(K, axis=1)
    D = np.diag(K_sum)

    return D


def make_L_matrix(K, D):
    D_temp = np.diag( np.diag(D) ** -0.5 )
    L = D_temp @ K @ D_temp
    
    w, v = LA.eig(L) # w = eigenvalues, v = normalized (unit “length”) eigenvectors
    
    return L

def step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    
    w = np.where(w >= lambda_cut, 1, 0)
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)
    
    return L_hat, D_hat, K_hat

def linear_step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    w = np.where(w >= lambda_cut, w, 0)
    
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)

    return L_hat, D_hat, K_hat

def polynomial_transfer(L, D, K, t):
    L_hat = L ** t
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ D**(1/2) @ (LA.inv(D) @ K)**t @ D**(1/2) @ D_hat**(1/2)
    K_hat = preprocessing.scale(K_hat)

    return L_hat, D_hat, K_hat
    

def apply_transfer_func(L, D, K, hyperparams, type="linear"):
    if type == "linear":
        return L, D, K
    if type == "step":
        k = hyperparams['k']
        return step_transfer(L, k)
    if type == "linear_step":
        k = hyperparams['k']
        return linear_step_transfer(L)
    if type == "polynomial":
        t = hyperparams['t']
        return polynomial_transfer(L, D, K, t)
        
    raise ValueError("wrong argument")


In [28]:

def accuracy(t, y):
    val = 0.0
    N = len(t)
    for i in range(N):
        if t[i] == y[i]:
            val += 1
    
    return val / N


def test_svm(X, Y, tf_fun, C=1, sigma=1 **kwargs):
    # Shuffle data
    np.random.seed(40)
    n_sample = len(X)
    order = np.random.permutation(n_sample)
    X = X[order]
    Y = Y[order].astype(np.float)
    
    # Make Kernel
    K = rbf_kernel(X, sigma)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K, kwargs, tf_fun)
    
    # Remove data without labels
    K_red = K[:70,:70]
    Y_red = Y[:70]
    
    K_test = K[70:100,:70]
    
    print(K_red.shape)
    
    # Apply to SVM
    clf = SVC(kernel="precomputed", C)
    clf.fit(K_red, Y_red)
    
    # Predict and accuracy
    y_pred = clf.predict(K_test)
    
    #print('t', Y[70:100])
    #print('y', y_pred)
    print("accuracy:", accuracy(y_pred, Y[70:100]))
    
    
    # Basic SVM
    clf2 = SVC(kernel="linear", C)
    clf2.fit(X[:70], Y_red)
    
    y_pred2 = clf2.predict(X[70:100])
    print("accuracy:", accuracy(y_pred2, Y[70:100]))

In [29]:
# load iris and split data
iris = load_iris()
X = iris.data[:,:2]
y = iris.target

X = X[y != 0]
y = y[y != 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

T = np.arange(1, 10)
for t in T:
    print("t = ", t)
    test_svm(X, y, "polynomial", t=t)

t =  1
(70, 70)
accuracy: 0.7
accuracy: 0.6
t =  2
(70, 70)
accuracy: 0.7
accuracy: 0.6
t =  3
(70, 70)
accuracy: 0.6333333333333333
accuracy: 0.6
t =  4
(70, 70)
accuracy: 0.6
accuracy: 0.6
t =  5
(70, 70)
accuracy: 0.6
accuracy: 0.6
t =  6
(70, 70)
accuracy: 0.5666666666666667
accuracy: 0.6
t =  7
(70, 70)
accuracy: 0.5
accuracy: 0.6
t =  8
(70, 70)
accuracy: 0.43333333333333335
accuracy: 0.6
t =  9
(70, 70)
accuracy: 0.43333333333333335
accuracy: 0.6


In [8]:




#Välkommen till våran_funktion()
'''
# find hyperparameters c and sigma with svm

best_result = 0
best_hyperparams = [best_c, best_sigma]

 for c in range(C)
    for sigma in range(Sigma)
        # do "runs" svms for stable average result
        
        data = shuffle(data)
        
        res = []
        for run in range(runs)
             res.append(svm(data, c, sigma))
        curr_avg_res /= runs
        
        if curr result is better than best_result:
            best_result = curr_result
            best_hyperparams = [c, sigma]        
            
            
            

'''

def find_hyperparameters():
    #split 10% of train dataset to use as validation
    
    c_values=np.arange(0.0, 10,0.1 )
    sigma_values=np.arange(0.01, 4,0.01 )
    print(c_values.shape)
    print(sigma_values.shape)
    
    lowest_error=1.0 #low is good
    best_parameter_values=[0,0]
    for c in c_values:
        for sigma in sigma_values:
            current_error=validate_hyperparameters(c,sigma)
            if (current_error<lowest_error):
                lowest_error=current_error
                best_parameter_values=c, sigma
    
    print(best_parameter_values,lowest_error)
def validate_hyperparameters(C, sigma):
    """Validates hyperparamters using k-fold cross validation with k=10"""
    
    #array that will contain error from each fold
    errors=[]
    
    #used for indexing in loop
    fold_size=int(X_train.shape[0]/10)
    
    for fold_n in range(10):
        #splits training data into 3 separate arrays 
        x_splits = np.split(
            X_train, [fold_n*fold_size, fold_n*fold_size+fold_size])
        #middle set is current validation set
        x_validation_set = x_splits[1]
        #merge first and second array from split to get training set
        x_training_set = np.append(x_splits[0], x_splits[2])

        
        #do same thing for y labels
        y_splits=np.split(
            y_train, [fold_n*fold_size, fold_n*fold_size+fold_size])
        y_validation_set = y_splits[1]
        y_training_set = np.append(y_splits[0], y_splits[2])
        
        #get error for current fold
        errors.append(get_svm_error(x_training_set,x_validation_set,y_training_set,y_validation_set,C,sigma))
    errors=np.array(errors)
    return errors.mean()

def get_svm_error(x_training_set,x_validation_set,y_training_set,y_validation_set,C,sigma):
    svm=SVC(kernel="rbf", C=C, gamma=sigma)

    svm.fit(x_training_set,y_training_set)
    y_pred=svm.predict(x_validation_set)
    error=1-accuracy(y_pred,y_validation_set)
    return error

    

# load iris and split data
iris = load_iris()
X = iris.data[:,:2]
y = iris.target

X = X[y != 0]
y = y[y != 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

find_hyperparameters()
    
    
    
    


(100,)
(399,)


ValueError: Expected 2D array, got 1D array instead:
array=[6.3 2.9 6.2 3.4 6.3 3.4 6.  3.  6.3 3.3 6.3 2.8 6.4 3.2 5.6 3.  6.  2.2].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.